# 1_Знакомимся с данными

✍ Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов JOIN.

?
А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим.

Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

In [1]:
# import image module
from IPython.display import Image

# get the image
Image(url="https://lms.skillfactory.ru/assets/courseware/v1/5903f8fada18b9da7c7c31ce8477feb6/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_1_1.jpg")

    Таблица city — это справочник городов. Структура справочника представлена ниже.

city_id	integer	уникальный идентификатор города, первичный ключ

city_name	text	название города

state	text	штат, к которому относится город

population	integer	население города

area	numeric	площадь города

    Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

cust_id	integer	уникальный идентификатор клиента, первичный ключ

cust_name	text	название клиента

annual_revenue	numeric	ежегодная выручка

cust_type	text	тип пользователя

address	text	адрес

zip	integer	почтовый индекс

phone	text	телефон

city_id	integer	идентификатор города, внешний ключ к таблице city

    Следующая таблица — driver — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

driver_id	integer	уникальный идентификатор водителя, первичный ключ

first_name	text	имя водителя

last_name	text	фамилия водителя

address	text	адрес водителя

zip_code	integer	почтовый индекс водителя

phone	text	телефон водителя

city_id	integer	идентификатор города водителя, внешний ключ к таблице city

    В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

truck_id	integer	Уникальный идентификатор грузовика, первичный ключ

make	text	Производитель грузовика

model_year	integer	Дата выпуска грузовика

    Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

ship_id	integer	уникальный идентификатор доставки, первичный ключ

cust_id	integer	идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer

weight	numeric	вес посылки

truck_id	integer	идентификатор грузовика, на котором отправлена доставка, внешний ключ
к таблице truck

driver_id	integer	идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver

city_id	integer	идентификатор города в который совершена доставка, внешний ключ
к таблице city

ship_date	date	дата доставки

# 2_UNION

## ПРИНЦИП И УСЛОВИЯ РАБОТЫ UNION

✍ Вернёмся к центральному вопросу модуля: как соединить несколько результатов, чтобы получить в выводе один общий?

Чтобы разобраться в этом вопросе, смоделируем ситуацию

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

SELECT book_name object_name, 'книга' object_descritption /*выбираем столбец с названием book_name, задаём алиас для столбца object_name, задаём во второй колонке объект ‘книга’ с алиасом для столбца object_descritption*/

FROM public.books /*из схемы public и таблицы books*/

UNION ALL /*оператор присоединения*/

SELECT movie_title, 'фильм' /*выбираем столбец movie_title, сами задаём во второй колонке объект ‘фильм’*/

FROM sql.kinopoisk /*из схемы sql и таблицы kinopoisk*/

Общий принцип мы поняли, разберёмся в деталях:

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:

* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

## ВИДЫ UNION

Оператор присоединения существует в двух вариантах:

* UNION выводит только уникальные записи;
* UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

    Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.

## СИНТАКСИС

Запрос строится таким образом:

SELECT

n columns

FROM 

table_1

UNION ALL

SELECT 

n columns

FROM 

table_2
...

UNION ALL

SELECT 

n columns

FROM 

table_n

Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.

    Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

Пришла пора испытать функцию UNION(ALL) на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.

SELECT

c.city_id object_name,  'id города' object_type /*выбираем колонку city_id и задаём ей алиас object_name, сами задаём объект 'id города' и название столбца object_type*/

FROM 

sql.city c /*из схемы sql и таблицы city, задаём алиас таблице — с*/

UNION ALL /*оператор присоединения*/

SELECT

d.driver_id other_name,  'id водителя' other_type /*выбираем колонку driver_id и задаём ей алиас other_name, сами задаём объект 'id водителя' и название столбца other_type*/

FROM 

sql.driver d  /*из схемы sql и таблицы driver, задаём алиас таблице — d*/

UNION ALL /*оператор присоединения*/

SELECT

s.ship_id,  'id доставки' /*выбираем колонку ship_id, сами задаём объект 'id доставки'*/

FROM 

sql.shipment s /*из схемы sql и таблицы shipment, задаём алиас таблице — s*/

UNION ALL /*оператор присоединения*/

SELECT

c.cust_id,  'id клиента' /*выбираем колонку cust_id, сами задаём объект 'id клиента'*/

FROM 

sql.customer c /*из схемы sql и таблицы customer, задаём алиас таблице — c*/

UNION ALL /*оператор присоединения*/

SELECT

t.truck_id,  'id грузовика' /*выбираем колонку truck_id, сами задаём объект 'id грузовика'*/

FROM 

sql.truck t /*из схемы sql и таблицы truck, задаём алиас таблице — t*/

ORDER BY 1 /*сортировка по первому столбцу*/

    Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.

    Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.

В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

SELECT book_name object_name, 'книга' object_descritption 

FROM public.books

UNION ALL

SELECT movie_title, 'фильм' 

FROM sql.kinopoisk

ORDER BY 1

LIMIT 1

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

SELECT book_name object_name, 'книга' object_descritption 

FROM public.books

ORDER BY 1

LIMIT 1

UNION ALL

SELECT movie_title, 'фильм' 

FROM sql.kinopoisk

ORDER BY 1

LIMIT 1

Вместо результата получим сообщение о синтаксической ошибке: "...syntax error at or near "UNION"..." Очевидно, этот фокус не удался.

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

(SELECT book_name object_name, 'книга' object_descritption 

FROM public.books

ORDER BY 1

LIMIT 1)

UNION ALL

(SELECT movie_title, 'фильм' 

FROM sql.kinopoisk

ORDER BY 1

LIMIT 1)

# 3_UNION и ограничение типов данных

## ПОЧЕМУ ТАК ВАЖЕН ТИП ДАННЫХ?

Как мы уже знаем, UNION может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Допустим, мы хотим вывести список всех id городов и их названий в одном столбце.

Давайте напишем запрос, который позволит получить нужный нам результат.

SELECT 

c.city_id /*выбираем столбец city_id*/

FROM

sql.city c /*из схемы sql  и таблицы city, задаём таблице алиас с*/ 

UNION ALL /*оператор присоединения*/

SELECT 

cc.city_name /*выбираем столбец city_name*/

FROM

sql.city cc /*из схемы sql и таблицы city, задаём таблице алиас сс*/

Вместо результата вы получите сообщение об ошибке: "ERROR: UNION types integer and text cannot be matched". Дело в том, что мы попытались объединить числовой и строковый типы в одной колонке, а это невозможно.

    Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в Postgres составляется запрос по модели column_name::column_type.

Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text.

    Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

Немного подправим запрос, чтобы получить желаемый результат.

SELECT 

c.city_id::text /*выбираем столбец city_id, переводим city_id из числового в текстовый формат*/

FROM

sql.city c /*из схемы sql  и таблицы city, задаём таблице алиас с*/

UNION ALL /*оператор присоединения*/

SELECT 

cc.city_name /*выбираем столбец city_name*/

FROM

sql.city cc /*из схемы sql и таблицы city, задаём таблице алиас сс*/

# 4_UNION ALL и промежуточные итоги

## ВОЗМОЖНОСТИ UNION

✍ Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.

Попробуем вывести обобщённые данные о населении по всем городам, с детализацией до конкретного города.

SELECT

c.city_name,

c.population /*выбираем столбцы city_name, population*/

FROM

sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/

UNION ALL /*оператор присоединения*/

SELECT

'total',

SUM(c.population) /*сами задаём объект ‘total’, суммируем все значения столбца population*/

FROM

sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/

ORDER BY 2 DESC /*сортируем по второму столбцу в убывающем порядке (чтобы итоговая сумма была в начале)*/

Визуально это действие можно представить так:

In [2]:
# import image module
from IPython.display import Image

# get the image
Image(url="https://lms.skillfactory.ru/assets/courseware/v1/5b38c2b9fa661c3e2c559506101f4fb7/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_4_1.png")

# 5_UNION и дополнительные условия

✍ UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

Например, с помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона.

SELECT

d.first_name,

d.last_name, 'телефон заполнен' phone_info /*выбираем столбцы first_name, last_name, сами выводим объект ‘телефон заполнен’*/

FROM

sql.driver d /*из схемы sql и таблицы driver, задаём алиас d*/

WHERE d.phone IS NOT NULL /*условие, что телефон заполнен*/

UNION /*оператор присоединения (уникальные значения)*/

SELECT

d.first_name,

d.last_name, 'телефон не заполнен' phone_info /*выбираем столбцы first_name, last_name, сами выводим объект ‘телефон не заполнен’*/

FROM

sql.driver d /*из схемы sql и таблицы driver, задаём алиас d*/

WHERE d.phone IS NULL /*условие, что телефон не заполнен*/

# 6_UNION и ручная генерация

✍ UNION можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.

SELECT 

'a' letter,'1' ordinal_position /*сами задаём значение первого столбца ‘a’ и алиас для него letter, значение второго столбца ‘1’ и алиас для него ordinal_position*/
         
UNION /*оператор присоединения*/

SELECT 

'b','2' /*сами задаём значение первого столбца ‘b’, значение второго столбца ‘2’ */
         
UNION /*оператор присоединения*/

SELECT

'c','3' /*сами задаём значение первого столбца ‘с’, значение второго столбца ‘3’*/ 

    Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы unicode и позиции элемента в ней с учётом определённых условий.

# 7_EXCEPT

## ИСКЛЮЧАЕМ ПОВТОРЯЮЩИЕСЯ ДАННЫЕ

Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.

SELECT

c.city_name /*выбираем столбец city_name*/

FROM

sql.shipment s /*из схемы sql и таблицы shipment, задаём таблице алиас s*/

JOIN sql.city c ON s.city_id = c.city_id /*внутреннее присоединение из схемы sql таблицы city, задав ей алиас c, по ключам city_id*/

EXCEPT /*оператор присоединения*/

SELECT

cc.city_name  /*выбираем столбец city_name*/

FROM

sql.driver d /*из схемы sql и таблицы driver, задаём таблице алиас d*/

JOIN sql.city cc ON d.city_id=cc.city_id /*внутреннее присоединение из схемы sql таблицы city, задав ей алияс cc, по ключам city_id*/

ORDER BY 1 /*сортировка по первому столбцу*/

Все водители проживают в городе Memphis, и мы видим, что он не выводится в результате запроса.

Чтобы лучше понять данный тип присоединения, предлагаем ознакомиться с диаграммой Венна — математическим инструментом, представляющим возможные логические связи между соединёнными наборами данных.

In [3]:
# import image module
from IPython.display import Image

# get the image
Image(url="https://lms.skillfactory.ru/assets/courseware/v1/fd51ef1b05d2fdc6f1c2e860cffb4cd7/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_7_1.png")

Синтаксические правила для оператора EXCEPT такие же, как и для UNION:

* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

Синтаксис выглядит следующим образом:

SELECT 

n columns

FROM 

table_1

EXCEPT

SELECT 

n columns

FROM 

table_2

    Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.

Предположим, у нас есть информация о продажах канцелярского магазина за май и июнь.

Какие-то позиции продавались и в том, и в другом месяце, а какие-то — только в одном. Использовав EXCEPT, мы можем оставить только те товары, которые есть в первом запросе (например, за май), но отсутствуют во втором запросе (например, за июнь).

Графически действие оператора можно представить следующим образом:

In [4]:
# import image module
from IPython.display import Image

# get the image
Image(url="https://lms.skillfactory.ru/assets/courseware/v1/37fa117a6e758a7af619e4732dd2b4bb/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_7_2.png")

Таким образом, при присоединении с помощью EXCEPT мы вывели только те товары, которые были проданы в мае, но не в июне. Чтобы найти продажи по тем позициям, что были реализованы в июне, а в мае — нет, необходимо поменять запросы местами.

# 8_INTERSECT

## ВЫБИРАЕМ ОБЩИЕ ДАННЫЕ

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах?

Предположим, нам надо вывести совпадающие по названию города и штаты.

SELECT 

c.city_name object_name /*выбираем столбец city_name, задаём ему алиас object_name*/

FROM 

sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/

INTERSECT /*оператор присоединения*/

SELECT 

cc.state /*выбираем столбец state*/

FROM 

sql.city cc /*из схемы sql и таблицы city, задаём таблице алиас с*/

ORDER BY 1

Как видим, с помощью оператора INTERSECT мы вывели названия городов и штатов, которые совпадают: New York, Washington и Wyoming. Присмотримся к нему внимательнее.

Чтобы лучше понять, как работает этот оператор, вновь обратимся к диаграмме Венна: INTERSECT оставляет из результатов первого запроса все строки, которые совпали с результатом выполнения второго запроса.

In [5]:
# import image module
from IPython.display import Image

# get the image
Image(url="https://lms.skillfactory.ru/assets/courseware/v1/5c4422f466caff0a493c3ec664658ae8/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_8_1.png")

Синтаксис запроса с оператором INTERSECT выглядит следующим образом:

SELECT 
    
n columns

FROM 

table_1

INTERSECT

SELECT 

n columns

FROM 

table_2

ернёмся к нашему примеру с продажами канцтоваров.

С помощью оператора INTERSECT мы можем вывести те позиции, которые продавались и в мае, и в июне. Визуализировать это действие можно примерно так:

In [6]:
# import image module
from IPython.display import Image

# get the image
Image(url="https://lms.skillfactory.ru/assets/courseware/v1/0b1111999cc786de32fe2480e80d139e/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u2-md4_8_2.png")

Оператор INTERSECT оставляет только те строки, которые являются общими для двух запросов (в нашем примере это Тетрадь).

    Как EXCEPT, так и INTERSECT убирают дубликаты, если они имеются.

# 9_Итоги. Закрепление знаний

✍ В этом модуле вы освоили ещё несколько возможностей из функционала SQL.

Теперь вы умеете:

присоединять таблицы «снизу», одну под другой;
исключать или, наоборот, выбирать повторяющиеся записи из двух таблиц;
вручную добавлять записи в таблицу.

Запишем структуру запроса с учётом полученных знаний.

SELECT 

N columns

FROM 

table_1

UNION / UNION ALL / EXCEPT / INTERSECT 

SELECT 

N columns

FROM 

table_2